In [ ]:
%pip install Recommenders 

     |████████████████████████████████| 233 kB 11.8 MB/s 
     |████████████████████████████████| 59 kB 6.0 MB/s 
     |████████████████████████████████| 93 kB 1.8 MB/s 
     |████████████████████████████████| 1.5 MB 65.7 MB/s 
     |████████████████████████████████| 11.8 MB 18.5 MB/s 
     |████████████████████████████████| 636 kB 65.4 MB/s 
     |████████████████████████████████| 61 kB 5.8 MB/s 
     |████████████████████████████████| 12.4 MB 41.3 MB/s 
     |████████████████████████████████| 2.6 MB 32.9 MB/s 
     |████████████████████████████████| 310 kB 72.2 MB/s 
     |████████████████████████████████| 3.3 MB 56.9 MB/s 
     |████████████████████████████████| 895 kB 85.5 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=697435 sha256=ca379b6ad239f6d2095a978e61d66e077aeea01890b43309cc55d1354c1050e6
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
  Created wheel for memory-profiler: f

In [1]:
import sys
#sys.path.append("../../")
import time
import os
import shutil
# import papermill as pm
import pandas as pd
import numpy as np
%tensorflow_version 1.x

import tensorflow as tf
#print(tensorflow.__version__)
from reco_utils.recommender.ncf.ncf_singlenode import NCF
from reco_utils.recommender.ncf.dataset import Dataset as NCFDataset
#from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split

from reco_utils.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)
from reco_utils.common.constants import SEED as DEFAULT_SEED

# top k items to recommend
TOP_K = 10

# Model parameters
EPOCHS = 100
BATCH_SIZE = 256

SEED = DEFAULT_SEED  # Set None for non-deterministic results


TensorFlow 1.x selected.


ModuleNotFoundError: ignored

,userID,itemID,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [ ]:
df2 = pd.read_csv("BX-Book-Ratings.csv",  delimiter="\;")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [ ]:
df2.columns=['userID','itemID','rating']

In [ ]:
df2.head()

,userID,itemID,rating
0,"""276725""","""034545104X""","""0"""
1,"""276726""","""0155061224""","""5"""
2,"""276727""","""0446520802""","""0"""
3,"""276729""","""052165615X""","""3"""
4,"""276729""","""0521795028""","""6"""


In [ ]:
for i, col in enumerate(df2.columns):
    df2.iloc[:, i] = df2.iloc[:, i].str.replace('"', '')

In [ ]:
df2.head()
df2['rating']=pd.to_numeric(df2['rating'])

In [ ]:

from reco_utils.dataset.python_splitters import python_random_split
# 75%비중으로 나누기 
train, test = python_random_split(df2, 0.75)

# user id, item id ,rating 가 있는 데이터만 가져옴 
data = NCFDataset(train=train, test=test, seed=SEED)

tf.compat.v1.disable_eager_execution()

In [ ]:
model = NCF(n_users=data.n_users,n_items=data.n_items,model_type="NeuMF",n_factors=4,layer_sizes=[16,8,4],\
            n_epochs=EPOCHS,batch_size=BATCH_SIZE,learning_rate=1e-3,verbose=10,seed=SEED)
# n_factors : latent space의 차원 수 (사용자 설정)
# layer_sizes: MLP 레이어 숫자 

# 모델학습 
model.fit(data)

predictions = [[row.userID, row.itemID, model.predict(row.userID, row.itemID)]
               for (_, row) in test.iterrows()]

# 예측값 저장하기 
predictions = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])
predictions.head()